In [120]:
import ssl
import xmlrpc
import xmlrpc.client
import base64
import datetime

import requests
import urllib3
import json

import requests
import pandas as pd

import random
import string
import time

In [121]:
from requests.adapters import HTTPAdapter

DEFAULT_TIMEOUT = 5 # seconds

class TimeoutHTTPAdapter(HTTPAdapter):
    def __init__(self, *args, **kwargs):
        self.timeout = DEFAULT_TIMEOUT
        if "timeout" in kwargs:
            self.timeout = kwargs["timeout"]
            del kwargs["timeout"]
        super().__init__(*args, **kwargs)

    def send(self, request, **kwargs):
        timeout = kwargs.get("timeout")
        if timeout is None:
            kwargs["timeout"] = self.timeout
        return super().send(request, **kwargs)

In [122]:
### POST FUNCTIONS
def PostToUdl(udl_endpoint, username, unecrypted_password, json_data, http_session):
    # Load auth
    key = username + ":" + unecrypted_password
    authkey = base64.b64encode(key.encode('utf-8')).decode("ascii")
    #creds = "Basic Y2hyaXN0aWFuLmtlbXBpczo5OTkwMCFDSUshY2lrISEh"

    udl_headers = {'accept': 'application/json',
                  'content-type': 'application/json',
                  #'Authorization': creds}
                  'Authorization': 'Basic {auth}'.format(auth=authkey)}
    print("Invoking {url} endpoint".format(url=udl_endpoint))
    print("calling with {data}".format(data=json_data))

    try:
        response = http_session.post( udl_endpoint, 
                                      data = json.dumps(json_data),
                                      verify = False,
                                      headers = udl_headers)
    except requests.exceptions.RequestException as e:
        print(f"ERROR ON POST:  {e}")

    if response.ok:
        print("Completed data access at {url}".format(url=udl_endpoint))

    return response

def PostTestVector(udl_ep, udl_user, udl_pw, sync_pattern, time_stamp, http_session):
    try:
        state_vector_data = {
            "classificationMarking": "U",
            "msgType" : "LatencyTest",
            "source" : "MITRE",
            "msgBody" : {   "timeStamp" : time_stamp, 
                            "message" : "System latency test", 
                            "syncPattern" : sync_pattern
                        },
            "dataMode": "TEST"
        }

        print('Start executing UDL update for ' + udl_user)
        res = PostToUdl(udl_ep, udl_user, udl_pw, state_vector_data, http_session)
        print(res)
        print('Finished executing UDL update...')
        return res
    except xmlrpc.client.ProtocolError as err:
        print("A protocol error occurred")
        print("URL: %s" % err.url)
        print("HTTP/HTTPS headers: %s" % err.headers)
        print("Error code: %d" % err.errcode)
        print("Error message: %s" % err.errmsg)

In [123]:
### QUERY FUNCTIONS
def GetFromUdl(url_endpoint, http_session, creds):
    try:
        response = http_session.get(url_endpoint, headers={'Authorization':creds}, verify=False)

    except requests.exceptions.RequestException as e:
        print(f"ERROR ON GET:  {e}")

    if response.ok:
        print("Completed data retrieval at {url}".format(url=url_endpoint))
        print(f"Elapsed Time on Get {response.elapsed}")

    return response


# Returns a single data frame with the matching sync pattern or -1 if non were found
def ScanDataFrameForOnSyncPattern(data_frame, sync_pattern):
    dfDecrementer = data_frame.shape[0] - 1    # Get amount of rows in the dataframe
    stopSearch = False
    returnValue = -1

    while stopSearch == False:
        dataRow = data_frame.loc[dfDecrementer]

        if 'syncPattern' in dataRow['msgBody']:
            compString = dataRow['msgBody']['syncPattern']

            if(compString == sync_pattern):
                print(f"Test Message Found for: {sync_pattern}");
                returnValue = dataRow
                stopSearch = True;

        dfDecrementer -= 1

        if(dfDecrementer < 0):
            print(f"Did not find message for : {sync_pattern}")
            stopSearch = True
            break
    
    return returnValue

# Returns time difference between the local timep stmap and retrieved data frame
def GetServerCreationLatency(time_stamp, single_data_frame):
    serverTime = datetime.datetime.strptime(single_data_frame['createdAt'],"%Y-%m-%dT%H:%M:%S.%fZ")
    givenTime = datetime.datetime.strptime(time_stamp,"%Y-%m-%dT%H:%M:%S.%fZ")
    
    timeDifference = serverTime - givenTime

    print(f"Local Time: {givenTime}, Server Created Time: {serverTime}")
    print(f"Total Time Difference: {timeDifference}")

    return timeDifference

def ConvertToMs(time):
    convert = str(round(time.total_seconds() * 1000)) + 'ms'
    return convert


In [124]:
# Load All credentials
credsData = 0
with open('creds.json') as creds:
    credsData = json.load(creds)

user = credsData['user']
password = credsData['password']
creds = credsData['creds']
serviceEndpointTest = credsData['service_endpoint_test']

# Load HTTP services
retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
retryTimeoutAdapter = TimeoutHTTPAdapter(max_retries=retries)

httpSession = requests.Session()
httpSession.mount("https://",retryTimeoutAdapter)
httpSession.mount("http://",retryTimeoutAdapter)

# Latency Checks
timeToPost = 0
timeToGet = 0
timeDiffOnServerCreate = 0





In [125]:
# Setup HTTP session

# POST TO SERVER
syncPattern = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
timeStamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
timeStamp = timeStamp + 'Z'

postResult = PostTestVector(serviceEndpointTest, user, password, syncPattern, timeStamp, httpSession)
timeToPost = ConvertToMs(postResult.elapsed)

Start executing UDL update for christian.kempis
Invoking https://pep.bluestaq.com/udl/notification/ endpoint
calling with {'classificationMarking': 'U', 'msgType': 'LatencyTest', 'source': 'MITRE', 'msgBody': {'timeStamp': '2020-04-01T23:19:38.564Z', 'message': 'System latency test', 'syncPattern': 'O3uMBZ7cs2ZE6sSz'}, 'dataMode': 'TEST'}
Completed data access at https://pep.bluestaq.com/udl/notification/
Elapsed Time on Post: 0:00:00.746221
<Response [201]>
Finished executing UDL update...


In [126]:
### BEGIN QUERY FROM DATABASE
getResult = GetFromUdl(serviceEndpointTest, httpSession, creds)
timeToGet = ConvertToMs(getResult.elapsed)
elsetsDataFrame = pd.DataFrame(getResult.json())

Completed data retrieval at https://pep.bluestaq.com/udl/notification/
Elapsed Time on Get 0:00:00.187663


In [127]:
singleDataFrame = ScanDataFrameForOnSyncPattern(elsetsDataFrame, syncPattern)
timeDiffOnServerCreate = ConvertToMs(GetServerCreationLatency(timeStamp, singleDataFrame))
print(f"POST TIME: {timeToPost}")
print(f"GET TIME:  {timeToGet}")
print(f"TIME DIFF: {timeDiffOnServerCreate}")



Test Message Found for: O3uMBZ7cs2ZE6sSz
Local Time: 2020-04-01 23:19:38.564000, Server Created Time: 2020-04-01 23:19:39.247000
Total Time Difference: 0:00:00.683000
POST TIME: 746ms
GET TIME:  188ms
TIME DIFF: 683ms
